## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-20-14-48-06 +0000,bbc,Palestinian woman in hospital after being club...,https://www.bbc.com/news/articles/cx20p0qx50no...
1,2025-10-20-14-45-49 +0000,bbc,Pizza Hut to close 68 UK restaurants,https://www.bbc.com/news/articles/c07mk59pzkpo...
2,2025-10-20-14-39-25 +0000,nypost,Bank robbers are turning blue state into hot s...,https://nypost.com/2025/10/20/us-news/bank-rob...
3,2025-10-20-14-39-09 +0000,nyt,"For Alaskan Evacuees, Home Is Gone, With No Re...",https://www.nytimes.com/2025/10/19/us/alaska-s...
4,2025-10-20-14-36-54 +0000,nyt,Rodrigo Paz Wins Bolivia’s Presidential Election,https://www.nytimes.com/2025/10/19/world/ameri...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2326/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
106,trump,25
81,louvre,14
269,new,11
150,gaza,11
129,fire,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
197,2025-10-19-16-10-25 +0000,bbc,Trump ends aid to Colombia and calls country's...,https://www.bbc.com/news/articles/cn8xg1jve73o...,67
185,2025-10-19-17-28-07 +0000,nyt,Trump Posts Fake A.I. Video of Himself Flying ...,https://www.nytimes.com/2025/10/19/us/politics...,65
24,2025-10-20-13-22-06 +0000,nypost,Trump insists cease-fire is holding in Gaza as...,https://nypost.com/2025/10/20/us-news/presiden...,63
120,2025-10-20-04-01-41 +0000,nypost,Kushner and Witkoff reveal how the Trump admin...,https://nypost.com/2025/10/20/world-news/kushn...,59
181,2025-10-19-17-32-48 +0000,nyt,Vance Hosts Marines Demonstration Amid ‘No Kin...,https://www.nytimes.com/2025/10/19/us/politics...,55


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
197,67,2025-10-19-16-10-25 +0000,bbc,Trump ends aid to Colombia and calls country's...,https://www.bbc.com/news/articles/cn8xg1jve73o...
149,52,2025-10-19-22-46-24 +0000,bbc,China will soon have a new Five Year Plan. Her...,https://www.bbc.com/news/articles/cjr045x425vo...
190,45,2025-10-19-17-06-24 +0000,nypost,Louvre robbery footage captures thief cutting ...,https://nypost.com/2025/10/19/world-news/louvr...
98,41,2025-10-20-09-00-00 +0000,nypost,Bombshell NYC mayoral poll shows trouble for Z...,https://nypost.com/2025/10/20/us-news/bombshel...
34,40,2025-10-20-12-48-54 +0000,bbc,Deadly Gaza flare-up tests Israel-Hamas ceasefire,https://www.bbc.com/news/articles/czxk8k4xlv1o...
87,34,2025-10-20-09-07-26 +0000,bbc,Two dead after cargo plane skids off Hong Kong...,https://www.bbc.com/news/articles/c5y0eeqz732o...
107,32,2025-10-20-07-04-22 +0000,nypost,Patrol car hit by shrapnel during Marine Corps...,https://nypost.com/2025/10/20/us-news/patrol-c...
181,28,2025-10-19-17-32-48 +0000,nyt,Vance Hosts Marines Demonstration Amid ‘No Kin...,https://www.nytimes.com/2025/10/19/us/politics...
40,24,2025-10-20-12-15-35 +0000,startribune,Amazon cloud computing outage disrupts Snapcha...,https://www.startribune.com/amazon-cloud-compu...
168,24,2025-10-19-19-27-11 +0000,nypost,Secret Service investigating possible sniper’s...,https://nypost.com/2025/10/19/us-news/secret-s...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
